In [1]:
from melee_dataset import MeleeDataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from torch.distributions import Categorical
from torch.distributions import Bernoulli,Normal
import matplotlib.pyplot as plt
from PolicyNet import PolicyNet
import torch.nn.functional as F
from Agents.BCAgent import BCAgent

%load_ext autoreload
%autoreload 2

In [2]:
# train_dataset = MeleeDataset(data_path="data/train_mini_515")
train_dataset = MeleeDataset(data_path="data/train_mini_100")
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [3]:
print(f"obs_dim: {train_dataset[0][0].shape}")
print(f"act_dim: {train_dataset[0][1].shape}")

Loaded Game_20250501T001638.slp with 15927 states and 15927 actions. The cache size is 1.
obs_dim: torch.Size([54])
act_dim: torch.Size([17])


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
obs_dim = train_dataset[0][0].shape[0]
act_dim = train_dataset[0][1].shape[0]
policy  = PolicyNet(obs_dim, act_dim).to(device)
# opt     = optim.Adam(policy.parameters(), lr=1e-3)
agent = BCAgent(obs_dim, act_dim, device)

Using device: cuda


In [5]:
losses = []
for epoch in range(6):
    total_loss = 0.0
    for i, (states, actions) in tqdm(enumerate(train_loader)):
        states = states.to(device)       # [B, obs_dim]
        actions = actions.to(device)     # [B] integers in [0…act_dim-1]

        loss = agent.train(states, actions)

        total_loss += loss * states.shape[0]
        losses.append(loss)
    avg_loss = total_loss / len(train_dataset)
    print(f"Epoch {epoch+1} — Loss: {avg_loss:.4f}")

0it [00:00, ?it/s]

Loaded Game_20250430T013632.slp with 16357 states and 16357 actions. The cache size is 2.
Loaded Game_20250430T134932.slp with 14271 states and 14271 actions. The cache size is 3.
Loaded Game_20250430T063350.slp with 10702 states and 10702 actions. The cache size is 4.
Loaded Game_20250430T110512.slp with 13286 states and 13286 actions. The cache size is 5.
Loaded Game_20250430T071642.slp with 13928 states and 13928 actions. The cache size is 6.
Loaded Game_20250430T051051.slp with 15510 states and 15510 actions. The cache size is 7.
Loaded Game_20250430T115152.slp with 12700 states and 12700 actions. The cache size is 8.
Loaded Game_20250430T231954.slp with 12677 states and 12677 actions. The cache size is 9.
Loaded Game_20250430T065555.slp with 14492 states and 14492 actions. The cache size is 10.
Loaded Game_20250430T081407.slp with 14347 states and 14347 actions. The cache size is 11.
Loaded Game_20250429T191041.slp with 7781 states and 7781 actions. The cache size is 12.
Loaded Ga

1it [01:58, 118.78s/it]

Loaded Game_20250430T075024.slp with 13711 states and 13711 actions. The cache size is 77.
Loaded Game_20250501T000340.slp with 8784 states and 8784 actions. The cache size is 78.
Loaded Game_20250430T035054.slp with 9790 states and 9790 actions. The cache size is 79.
Loaded Game_20250430T001724.slp with 8446 states and 8446 actions. The cache size is 80.
Loaded Game_20250430T093627.slp with 11722 states and 11722 actions. The cache size is 81.
Loaded Game_20250501T000951.slp with 8246 states and 8246 actions. The cache size is 82.
Loaded Game_20250430T224332.slp with 12456 states and 12456 actions. The cache size is 83.
Loaded Game_20250430T163733.slp with 15837 states and 15837 actions. The cache size is 84.
Loaded Game_20250430T042403.slp with 12189 states and 12189 actions. The cache size is 85.
Loaded Game_20250430T234610.slp with 7291 states and 7291 actions. The cache size is 86.
Loaded Game_20250429T200904.slp with 11780 states and 11780 actions. The cache size is 87.
Loaded Ga

2it [02:26, 65.51s/it] 

...

Epoch 1 — Loss: -22.4876


9072it [00:20, 435.21it/s]


Epoch 2 — Loss: -27.5125


9072it [00:20, 437.43it/s]

Epoch 3 — Loss: -119.3927


In [ ]:
# Save the trained model
torch.save(policy.state_dict(), "trained_policy_distribution_combined.pth")
print("Model saved to trained_policy.pth")


Model saved to trained_policy.pth


: 